In [21]:
import pandas as pd
import numpy as np
import re
from gensim.models import word2vec, Word2Vec
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np
from numpy import argmax
%matplotlib inline

from ckonlpy.tag import Twitter #konlpy twitter는 add_dictionary가 없어서, ckonlpy(customized konlpy) 사용.
twitter = Twitter()

C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [39]:
df = pd.read_csv('connector_1.csv', sep='\t', encoding='UTF-8')

## 텍스트 전처리

In [40]:
# 예측값과 실제값 비교를 위해 컬럼을 하나 더 생성합니다.
df['result_pred'] = df['검토결과'].astype(str).copy()
# df['target_pred'] = df['대상 선정'].astype(str).copy()
# df['charge_pred'] = df['책임구'].astype(str).copy()
# df['division_pred'] = df['구분'].astype(str).copy()

### (1) 오타, 동의어 처리(replace)

replaceword = pd.read_csv('replaceword_final.csv', sep='\t', encoding='UTF-8')

replaceword.head(10)

replaceword.shape

df['clm_prtl_sbc'][488]

Replaceword_list =  replaceword['Replaceword'].tolist()

Missword_list = replaceword['Missword'].tolist()

print(len(Replaceword_list))
print(len(Missword_list))

for i in range(len(Replaceword_list)):
    df['clm_prtl_sbc'] = df['clm_prtl_sbc'].replace(Missword_list[i], Replaceword_list[i], regex=True)
#regex=True 설정을 주면 조사가 붙어있던 말던 단어를 다 바꿔준다.

df['clm_prtl_sbc'].head()

df['clm_prtl_sbc'][488] # '간극'이 '간격'으로 바뀌었는지 확인.(8529 데이터 기준)

df['clm_prtl_sbc'][670]  # '가니시'가 '가니쉬'로 바뀌었는지 확인.(8529 데이터 기준)

### (2) 사용자 단어 정의(User dictionary)

udw = pd.read_csv('Userdict_final.csv', sep='\t', encoding='UTF-8') #Userdictionary에 등록할 Word 불러오기.

udw.head()

udw_list =  udw['Userdict'].tolist() # 단어들만 list로 추출.

for word in udw_list: 
    twitter.add_dictionary(word, 'Noun') # for문을 통해 dictionary에 단어들을 Noun으로 추가.

twitter.pos(df['clm_prtl_sbc'][1]) # '리어', '미장착'이 Noun으로 잘 정의되었는지 확인.

### (3) 명사 추출(Twitter)

def nouns(text):
    text = str(text)
    text = twitter.nouns(text)
    return text

df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(nouns)

df.loc[1, 'clm_prtl_sbc']  #'리어', '미장착' 붙어서 출력되었는지 확인.(8529 데이터 기준)

### (4) 특수문자, 띄어쓰기 전처리

def preprocessing(text):
    # 개행문자 제거
    text= str(text)
    text = re.sub('\\\\n', ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    #text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ]', ' ', text) 
    text = re.sub(' +', ' ', text)
    return text

# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ['(LH)','(RH)','(사진첨부)','(우)','(좌)','[C]','A/S','C','C-10000','C-10000','F/LH','F/RH','FR','FRT',
             'Frt','frt','L/H','L/H리어','LH','LH/RH','LH쪽','R/H','R/H리어','R/LH','R/RH','RH','RH쪽','Rr','RR','rr',
             '결과','고객','고객에게','공임','광주','그랜저','그랜져','낫','낮','넥쏘','니로','당','당차량','당차량은','대리점',
             '동영상참조','레이','렌트','리무진','리어LH','리어RH','모닝','모하비','밤','벨로스터','사진','사진참조','사진첨부',
             '상기','상기차량은','새벽','새차','서비스','소하','스타렉스','스팅어','스포티지','신차','신차','심야','싼타페',
             '써비스','쏘나타','쏘렌토','쏘울','아반떼','아산','아이오닉','아침','야간','언덕','엑센트','열','울산','원인',
             '임시 넘버','임시 번호','입고','자동차','저머검내용','점검/원인','점검결과','점검내용','점검및원인','점검및조치내용',
             '점검점검','점검점검내용','조치결과','조치내용','주간','주재원','주제원','주차장','증상','지하','진입','차량',
             '차량은','철도','첨부','첨부파일','청구','출고','카 마스터','카니발','카렌스','코나','클레임','투싼',
             '팰리세이드','하이리무진','하이브리드','현     상','현    상','현   상','현  상','현 상','현산','현상','현상및점검',
             '현상및조치','화성'
]
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words) #stops에 안 걸리는 나머지 것들 이어붙이기

%time df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(preprocessing)

%time df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(remove_stopwords)

df['clm_prtl_sbc'].head()

474
474
Wall time: 78 ms
Wall time: 187 ms


0     인도 후 오디오 작동 시 주파수 및 이음 발 생으로 불량 제기 점검 내용 및 점검...
1     주행 중 엔진 경고등 점등 점검 내용 및 스캔 점검 효율 저하 발생 임시 코드 주...
2                   조수석 사이드 어백 경고등 점등 커넥터 터 체결 불량 재체결 
3                               작동 불량 커넥터 터 조립 불량 재조립 
4                             어라운드뷰 장착 운전석 미러 카메라 미장착 
Name: clm_prtl_sbc, dtype: object

## X, y 정의

In [41]:
from sklearn.preprocessing import LabelEncoder
y_encod = df['검토결과']
classle = LabelEncoder()
y_encod = classle.fit_transform(df['검토결과'].values) # y값 인코딩.
print(np.unique(y_encod))

# ro column을 list형태로 ro_text 변수에 할당.
ro_text = list(df['clm_prtl_sbc'])
sentiments = list(y_encod) # y값.

ro_text[0]

sentiments[:10]

sentences = []
for text in ro_text:
    sentences.append(text.split())

[0 1 2 3 4]


## Word2Vec

In [42]:
# Word2Vec의 파라미터 값들 정의.
num_features = 300    
min_word_count = 40   
num_workers = 4       
context = 10          
downsampling = 1e-3 

In [43]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
   level=logging.INFO)

In [44]:
# word2vec으로 단어 학습.
from gensim.models import word2vec

model = word2vec.Word2Vec(sentences, workers=num_workers, \
           size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

2019-06-04 08:14:22,828 : INFO : collecting all words and their counts
2019-06-04 08:14:22,829 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-06-04 08:14:22,837 : INFO : collected 2625 word types from a corpus of 49922 raw words and 1601 sentences
2019-06-04 08:14:22,838 : INFO : Loading a fresh vocabulary
2019-06-04 08:14:22,840 : INFO : effective_min_count=40 retains 185 unique words (7% of original 2625, drops 2440)
2019-06-04 08:14:22,840 : INFO : effective_min_count=40 leaves 38548 word corpus (77% of original 49922, drops 11374)
2019-06-04 08:14:22,841 : INFO : deleting the raw counts dictionary of 2625 items
2019-06-04 08:14:22,842 : INFO : sample=0.001 downsamples 84 most-common words
2019-06-04 08:14:22,843 : INFO : downsampling leaves estimated 18571 word corpus (48.2% of prior 38548)
2019-06-04 08:14:22,844 : INFO : estimated required memory for 185 words and 300 dimensions: 536500 bytes
2019-06-04 08:14:22,845 : INFO : resetting layer weight

In [45]:
# Word2Vec의 feature vector를 얻는 함수.
def get_features(words, model, num_features):
    feature_vector = np.zeros((num_features),dtype=np.float32)

    num_words = 0
    index2word_set = set(model.wv.index2word)

    for w in words:
        if w in index2word_set:
            num_words += 1
            feature_vector = np.add(feature_vector, model[w])

    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [46]:
def get_dataset(ro_text, model, num_features):
    dataset = list()

    for s in ro_text:
        dataset.append(get_features(s, model, num_features))

    reviewFeatureVecs = np.stack(dataset)
    
    return reviewFeatureVecs

In [47]:
test_data_vecs = get_dataset(sentences, model, num_features)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [48]:
test_data_vecs

array([[-8.27847570e-02, -9.79045779e-03,  6.82138950e-02, ...,
        -5.11349104e-02,  1.50814474e-01, -1.15143694e-01],
       [-9.16715786e-02,  2.54333373e-02,  9.66782495e-02, ...,
        -8.26524049e-02,  7.34057799e-02, -1.13863170e-01],
       [-7.20813274e-02, -2.49113403e-02,  7.50372484e-02, ...,
        -4.84125055e-02,  1.54924437e-01, -1.16220497e-01],
       ...,
       [-6.47724569e-02, -7.86865726e-02,  3.41362730e-02, ...,
        -1.59113610e-04,  2.30794519e-01, -1.27823979e-01],
       [-9.39836204e-02,  1.98074598e-02,  9.79037583e-02, ...,
        -8.13700184e-02,  8.39315802e-02, -1.17358774e-01],
       [-9.23976377e-02,  9.54447314e-03,  8.88261870e-02, ...,
        -7.28190467e-02,  1.04796812e-01, -1.19950093e-01]], dtype=float32)

## Train, Test data Split

In [49]:
from sklearn.model_selection import train_test_split
import numpy as np

X = test_data_vecs
y = np.array(sentiments)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 모델학습 및 평가

In [50]:
# pip install xgboost로 설치
import xgboost as xgb

# XGBoost Parameters http://xgboost.readthedocs.io/en/latest/parameter.html
xgb = xgb.XGBClassifier(n_estimators=10,
                          max_depth=6,
                          learning_rate=1.0,
                          max_delta_step=1,
                          eta = 1,
                          nthread=-1,
                          seed=2018)
xgb

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eta=1, gamma=0, learning_rate=1.0,
       max_delta_step=1, max_depth=6, min_child_weight=1, missing=None,
       n_estimators=10, n_jobs=1, nthread=-1, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=2018, silent=True, subsample=1)

In [51]:
%time xgb.fit(X_train, y_train)

Wall time: 743 ms


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eta=1, gamma=0, learning_rate=1.0,
       max_delta_step=1, max_depth=6, min_child_weight=1, missing=None,
       n_estimators=10, n_jobs=1, nthread=-1, objective='multi:softprob',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=2018, silent=True, subsample=1)

In [52]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

scoring = 'accuracy'
%time score = cross_val_score(xgb, X_train, y_train, cv=k_fold, n_jobs=-1, scoring=scoring)
score

Wall time: 7.41 s


array([0.84375   , 0.875     , 0.82421875, 0.875     , 0.8203125 ])

In [53]:
round(np.mean(score)*100,2)

84.77

In [54]:
from sklearn.metrics import accuracy_score
# 테스트 데이터를 넣고 예측합니다.
y_pred = xgb.predict(X_test)
y_pred[:3]

array([0, 3, 0])

In [55]:
print(accuracy_score(y_test, y_pred))

0.8722741433021807
